In [ ]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.5 MB/s eta 0:00:00


In [ ]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00


In [ ]:
train_arg = True
startDate = '2021-01-01'
endDate = '2021-02-01'

import torch
import pandas as pd
import datetime
import numpy as np
from datasets import Dataset
from sklearn.model_selection import train_test_split

# from accelerate import Accelerator
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, AdamW, get_scheduler
from torch.utils.data import DataLoader

from time import time

# TRAIN OR EVAL
train = train_arg
trained_model_name = "model_covid_twitter_bert_v2.model"

# MODEL DEFINITION
if train:
    # BASE_MODEL = "digitalepidemiologylab/covid-twitter-bert"
    # BASE_MODEL = "./covid-twitter-bert"
    BASE_MODEL = "digitalepidemiologylab/covid-twitter-bert-v2"
    # BASE_MODEL = "./covid-twitter-bert-v2"
else:
    BASE_MODEL = "/content/" + trained_model_name
LEARNING_RATE = 2e-5
MAX_LENGTH = 512
# BATCH_SIZE = 16
BATCH_SIZE = 2
EPOCHS = 3

# accelerator = Accelerator()

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
model = AutoModelForSequenceClassification.from_pretrained(BASE_MODEL, num_labels=1)
optimizer = AdamW(model.parameters(), lr=3e-5)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

print(torch.cuda.is_available())

print(torch.cuda.device_count())

print(torch.cuda.current_device())

print(torch.cuda.device(0))

print(torch.cuda.get_device_name(0))

# DATA PREPARATION

g_cases_filename = f"/content/data/g_cases_2021.csv"
y = pd.read_csv(g_cases_filename)

X = pd.DataFrame()
# date_ranges = [['2021-01-01', '2021-04-01']]
# date_ranges = [['2020-01-01', '2020-01-12']]
date_ranges = [[startDate, endDate]]
for date_range in date_ranges:
    start = datetime.datetime.strptime(date_range[0], "%Y-%m-%d")
    end = datetime.datetime.strptime(date_range[1], "%Y-%m-%d")
    date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end - start).days)]

    for date in date_generated:
        date_str = date.strftime("%Y-%m-%d")

        filename = f"/content/data/en_{date_str}_output.csv"
        df_data = pd.read_csv(filename)
        df_data = df_data.replace(r'^\s*$', np.nan, regex=True)
        #X = total_dataframe.append(df_data, ignore_index=True)
        X = pd.concat([X, df_data], ignore_index=True)

print(f"X.shape: {X.shape}")
print(f"X.head(): {X.head()}")

train_ratio = 0.70
validation_ratio = 0.15
test_ratio = 0.15

# train is now 70% of the entire data set, test size 30%
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_ratio, random_state=42, shuffle=False)
#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, train_size=0.7, random_state=42, shuffle=False)

# test is now 15% of the initial data set
# validation is now 15% of the initial data set
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio), random_state=42, shuffle=False)
train_df = pd.concat([X_train, y_train], axis=1)
val_df = pd.concat([X_val, y_val], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

print("train_df")
print(f"train_df.shape: {train_df.shape}")
print(train_df)

print("val_df")
print(f"val_df.shape: {val_df.shape}")
print(val_df)

print("test_df")
print(f"test_df.shape: {test_df.shape}")
print(test_df)

raw_train_ds = Dataset.from_pandas(train_df)
raw_val_ds = Dataset.from_pandas(val_df)
raw_test_ds = Dataset.from_pandas(test_df)

max_length = tokenizer.model_max_length
print("Maximum input sequence length:", max_length)
# Maximum input sequence length: 1000000000000000019884624838656

ds = {"train": raw_train_ds, "validation": raw_val_ds, "test": raw_test_ds}

print("ds")
print(ds)

# CALCULATE MAX_LENGTH
# Tokenize all tweets to find the maximum tokenized length
# max_length = 0

# for index, row in X.iterrows():
#     tweet = row['clean_tweets']

#     # Tokenize the tweet
#     tokens = tokenizer(tweet, return_tensors="pt")

#     # Get the length of the tokenized sequence
#     length = tokens['input_ids'].shape[1]
#     print(f"current length: {length}")

#     # Update max_length if needed
#     if length > max_length:
#         max_length = length

# print(f"Calculated max_length: {max_length}")
# Model's max length
# max_length = 512
# RuntimeError: The size of tensor a (600) must match the size of tensor b (512) at non-singleton dimension 1

def preprocess_function(examples):
    label = examples["g_values"]
    # examples = tokenizer(examples["clean_tweets"], truncation=True, padding="max_length", max_length=256)
    # examples = tokenizer(examples["clean_tweets"], truncation=False, padding=True, return_tensors="pt")
    examples = tokenizer(examples["clean_tweets"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
    # examples = tokenizer(examples["clean_tweets"], truncation=True, padding="max_length")

    # examples = tokenizer(examples["clean_tweets"])

    examples["label"] = float(label)
    # examples["label"] = [float(i) for i in label]
    print(examples)
    return examples

from torch.utils.data import DataLoader

if train:
    ds["train"] = ds["train"].map(preprocess_function, remove_columns=["__index_level_0__", "date", "total_cases", "g_values", "created_at", "clean_tweets"])
    ds["validation"] = ds["validation"].map(preprocess_function, remove_columns=["__index_level_0__", "date", "total_cases", "g_values", "created_at", "clean_tweets"])

    train_dataloader = DataLoader(
        ds["train"], shuffle=False, batch_size=BATCH_SIZE, collate_fn=data_collator
    )
    eval_dataloader = DataLoader(
        ds["validation"], batch_size=BATCH_SIZE, collate_fn=data_collator
    )
else:
    ds["test"] = ds["test"].map(preprocess_function, remove_columns=["__index_level_0__", "date", "total_cases", "g_values", "created_at", "clean_tweets"])

    test_dataloader = DataLoader(
        ds["test"], batch_size=BATCH_SIZE, collate_fn=data_collator
    )

# for split in ds:
    # ds[split] = ds[split].map(preprocess_function, remove_columns=["__index_level_0__", "date", "total_cases", "g_values", "created_at", "clean_tweets"])

print("ds")
print(ds)

for batch in train_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    break
print({k: v.shape for k, v in batch.items()})

outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

def training():
    num_training_steps = EPOCHS * len(train_dataloader)
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps,
    )
    print(f"num_training_steps: {num_training_steps}")

    from tqdm.auto import tqdm

    progress_bar = tqdm(range(num_training_steps))

    model.train()
    for epoch in range(EPOCHS):
        for batch_idx, batch in enumerate(train_dataloader):
            print(f"epoch: {epoch}, batch: {batch_idx}")
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            loss.backward()
            # accelerator.backward(loss)

            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(1)

def evaluation(dataloader):
    eval_preds = []
    eval_labels = []
    model.eval()
    for batch in dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        print(f"outputs['logits'].shape: {outputs['logits'].shape}")
        eval_preds.append(outputs['logits'])

        print(f"batch['labels'].shape: {batch['labels'].shape}")
        eval_labels.append(batch['labels'])

    eval_preds = torch.cat(eval_preds, dim=0)
    print(f"eval_preds: {eval_preds}")
    eval_labels = torch.cat(eval_labels, dim=0)
    print(f"eval_labels: {eval_labels}")
    metrics = compute_metrics_for_regression(eval_preds, eval_labels)
    print(metrics)

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

def compute_metrics_for_regression(eval_pred, eval_labels):
    print(f"eval_pred: {eval_pred}")
    print(f"eval_labels: {eval_labels}")
    logits = eval_pred
    labels = eval_labels
    labels = torch.reshape(labels, (-1, 1))
    print(f"labels: {labels}")

    mse = mean_squared_error(labels.cpu(), logits.cpu())
    mae = mean_absolute_error(labels.cpu(), logits.cpu())
    r2 = r2_score(labels.cpu(), logits.cpu())
    single_squared_errors = ((logits.cpu() - labels.cpu()).flatten()**2).tolist()

    # Compute accuracy
    # Based on the fact that the rounded score = true score only if |single_squared_errors| < 0.5
    accuracy = sum([1 for e in single_squared_errors if e < 0.25]) / len(single_squared_errors)

    return {"mse": mse, "mae": mae, "r2": r2, "accuracy": accuracy}

if train:
    ## TRAINING
    t = time()

    training()
    evaluation(eval_dataloader)
    model.save_pretrained(trained_model_name, from_pt=True)
    tokenizer.save_pretrained(trained_model_name)

    print('Time to train model: {} mins'.format(round((time() - t) / 60, 2)))

else:
    ## EVALUATION
    t = time()
    evaluation(test_dataloader)

    print('Time to eval model: {} mins'.format(round((time() - t) / 60, 2)))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at digitalepidemiologylab/covid-twitter-bert-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


True
1
0
Tesla T4
X.shape: (31, 2)
X.head():    created_at                                       clean_tweets
0  2021-01-01  mf doom passed favourite shop covid casualty i...
1  2021-01-02  vocal role head covid task force told people w...
2  2021-01-03  love grandma burdensome think people overweigh...
3  2021-01-04  covid leave hospital facing staffing issue sta...
4  2021-01-05  covid protocol net star unavailable week detai...
train_df
train_df.shape: (21, 5)
    created_at                                       clean_tweets        date  \
0   2021-01-01  mf doom passed favourite shop covid casualty i...  2021-01-01   
1   2021-01-02  vocal role head covid task force told people w...  2021-01-02   
2   2021-01-03  love grandma burdensome think people overweigh...  2021-01-03   
3   2021-01-04  covid leave hospital facing staffing issue sta...  2021-01-04   
4   2021-01-05  covid protocol net star unavailable week detai...  2021-01-05   
5   2021-01-06  sod covid denier economic fore

Map:   0%|          | 0/21 [00:00<?, ? examples/s]

{'input_ids': [101, 1049, 2546, 12677, 2979, 8837, 4497, 2522, 17258, 19844, 5665, 4769, 16405, 2015, 5665, 4119, 2522, 17258, 4737, 2522, 17258, 11891, 6887, 27292, 6305, 2923, 4727, 9846, 9998, 2522, 17258, 17404, 6603, 17752, 3039, 3347, 4825, 2095, 6574, 5899, 2057, 3726, 2136, 6325, 2344, 4994, 2248, 2739, 4034, 6838, 20710, 4939, 2351, 2522, 17258, 2995, 2995, 3246, 16360, 6175, 20710, 4939, 20735, 3519, 4352, 4148, 6926, 3793, 9359, 2522, 17258, 4013, 2522, 17258, 2317, 2160, 4366, 8398, 22301, 3477, 3021, 3477, 4189, 3745, 5023, 4171, 11633, 3129, 2987, 2102, 4714, 2192, 2377, 5439, 8971, 2522, 17258, 8971, 2449, 6090, 3207, 7712, 25537, 19002, 7309, 8211, 2439, 3105, 3318, 3036, 5738, 16393, 16582, 24501, 26029, 5332, 6321, 4497, 24447, 2490, 4189, 2121, 5929, 3068, 3647, 3407, 2095, 2522, 17258, 17710, 2063, 7475, 5292, 11335, 4757, 3648, 9871, 7309, 4287, 3407, 17207, 2480, 7205, 2902, 6951, 2522, 17258, 3131, 4148, 24955, 2522, 17258, 6526, 2711, 2439, 3105, 2711, 2351, 252